<a href="https://colab.research.google.com/github/Bambika69/DeepReinforcementLearningHuggingface/blob/main/A2C_PandaReachDense_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
# Install the specific setuptools version required to install the dependencies
!pip install setuptools==65.5.0

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit6/requirements-unit6.txt

In [ ]:
import pybullet_envs
import panda_gym
import gym

import os

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [ ]:
env_id = "PandaReachDense-v2"

# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape
a_size = env.action_space

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())  # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())  # Take a random action

In [ ]:
# Define the environment called “PandaReachDense-v2” & make a vectorized environment
env_id = "PandaReachDense-v2"
env = make_vec_env(env_id, n_envs=4)

# Add a wrapper to normalize the observations and rewards.
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.0)

# Create the A2C Model (don’t forget verbose=1 to print the training logs)
model = A2C(
    policy="MultiInputPolicy",
    env=env,
    gae_lambda=0.9,
    gamma=0.99,
    learning_rate=0.00096,
    max_grad_norm=0.5,
    n_steps=8,
    vf_coef=0.4,
    ent_coef=0.0,
    policy_kwargs=dict(log_std_init=-2, ortho_init=False),
    normalize_advantage=False,
    use_rms_prop=True,
    use_sde=True,
    verbose=1,
)

Using cuda device


In [ ]:
# Train it for 1M Timesteps
model.learn(1_500_000)

A streamkimeneten csak az utolsó 5000 sor látható.
| time/                 |          |
|    fps                | 336      |
|    iterations         | 19100    |
|    time_elapsed       | 1818     |
|    total_timesteps    | 611200   |
| train/                |          |
|    entropy_loss       | 7.78     |
|    explained_variance | 0.976    |
|    learning_rate      | 0.00096  |
|    n_updates          | 19099    |
|    policy_loss        | 0.0429   |
|    std                | 0.0385   |
|    value_loss         | 0.000296 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 50       |
|    ep_rew_mean        | -0.971   |
| time/                 |          |
|    fps                | 336      |
|    iterations         | 19200    |
|    time_elapsed       | 1827     |
|    total_timesteps    | 614400   |
| train/                |          |
|    entropy_loss       | 8.52     |
|    explained_variance 

In [ ]:
notebook_login()
!git config --global credential.helper store

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Save the model and VecNormalize statistics when saving the agent
model_name = "a2c-PandaReachDense-v2"
model.save(model_name)
env.save("vec_normalize.pkl")

# Evaluate agent
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v2")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load(model_name)

mean_reward, std_reward = evaluate_policy(model, env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Bandika/a2c-{env_id}", 
    commit_message="Initial commit",
)

Mean reward = -0.46 +/- 0.14
ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/panda_gym/pybullet.py:97: UserWarning: The use of the render method is not recommended when the environment has not been created with render=True. The rendering will probably be weird. Prefer making the environment with option `render=True`. For example: `env = gym.make('PandaReach-v2', render=True)`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/panda_gym/pybullet.py:97: UserWarning: The use of the render method is not recommended when the environment has not been created with render=True. The rendering will probably be weird. Prefer making the environment with option `render=True`. For example

Saving video to /tmp/tmpjrfl4x4y/-step-0-to-step-1000.mp4


Exception ignored in: <function VecVideoRecorder.__del__ at 0x7f8712eff4c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/vec_env/vec_video_recorder.py", line 112, in __del__
    self.close()
  File "/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/vec_env/vec_video_recorder.py", line 108, in close
    VecEnvWrapper.close(self)
  File "/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py", line 279, in close
    return self.venv.close()
  File "/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py", line 279, in close
    return self.venv.close()
  File "/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/vec_env/dummy_vec_env.py", line 80, in close
    env.close()
  File "/usr/local/lib/python3.9/dist-packages/gym/core.py", line 298, in close
    return self.env.close()
  File "/usr/local/lib/python3.9/dist-packages/panda_gym/envs

ℹ Pushing repo Bandika/a2c-PandaReachDense-v2 to the Hugging Face
Hub


...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

policy.pth:   0%|          | 0.00/46.7k [00:00<?, ?B/s]

a2c-PandaReachDense-v2.zip:   0%|          | 0.00/110k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Bandika/a2c-PandaReachDense-v2/tree/main/


'https://huggingface.co/Bandika/a2c-PandaReachDense-v2/tree/main/'